In [1]:
!pip install vaderSentiment

In [2]:
#import modules
import numpy as np
from math import sqrt


#nlp text processing dependencies
import re
import string
import inflector
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

#sentiment analysis libraray
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

#pyspark dependencies
import pyspark
from pyspark.sql.functions import *
from pyspark.ml.clustering import KMeans
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SQLContext
from pyspark.ml.feature import Word2Vec

from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, FloatType, DoubleType, IntegerType


In [3]:
#define spark context
spark = SparkSession\
        .builder\
        .appName("SentimentAnalysis")\
        .getOrCreate()

sc = spark.sparkContext


In [4]:
sqlContext = SQLContext(sc)

In [5]:
# load csv data
raw_dataframe = spark.read.format("csv") \
.option("header", "true") \
.load("/Users/agarwalshruti28/Downloads/Twitter\ Analysis/coronavirus-covid19-tweets/2020-03-12\ Coronavirus\ Tweets.CSV")

#print Schema
raw_dataframe.printSchema()


root
 |-- status_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- reply_to_status_id: string (nullable = true)
 |-- reply_to_user_id: string (nullable = true)
 |-- reply_to_screen_name: string (nullable = true)
 |-- is_quote: string (nullable = true)
 |-- is_retweet: string (nullable = true)
 |-- favourites_count: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- place_full_name: string (nullable = true)
 |-- place_type: string (nullable = true)
 |-- followers_count: string (nullable = true)
 |-- friends_count: string (nullable = true)
 |-- account_lang: string (nullable = true)
 |-- account_created_at: string (nullable = true)
 |-- verified: string (nullable = true)
 |-- lang: string (nullable = true)



In [6]:
#register temp table for sql usage
raw_dataframe.registerTempTable("raw_dataframe")

In [7]:

# filter only english tweets and get status_id and text for pr
english_tweets_raw = spark.sql("""SELECT status_id, text

                     FROM raw_dataframe
                     
                     WHERE lang="en"
                   """ )



In [8]:
# top 20 rows
english_tweets_raw.show()

+-------------------+--------------------+
|          status_id|                text|
+-------------------+--------------------+
|1237891051307896832|Tonight: Wednesda...|
|1237891052159119366|No, but seriously...|
|1237891052779884545|To help those in ...|
|1237891053182738432|Today in the ente...|
|1237891053686075392|This JAMA Insight...|
|1237891054600241152|Watch this newly ...|
|1237891054721880067|Think about alter...|
|1237891054797352964|With 200 colleges...|
|1237891055199981578|Get the latest on...|
|1237891055502020612|Do you trust your...|
|1237891060879118337|Shocking X-ray im...|
|1237891072358903813|This is a stagger...|
|1237891073503944705|Fucking moron #Wo...|
|1237891074539974665|University of Cop...|
|1237891075445903363|5 Pack Face Masks...|
|1237891075970412545|Aye @realDonaldTr...|
|1237891078105284608|Small gadget to f...|
|1237891079057190913|On https://t.co/p...|
|1237891085281546240|As a mom, I’ve ca...|
|1237891087764619266|$ALT -IMO with on...|
+----------

In [9]:
#total number of english tweets
english_tweets_raw.count()

167449

In [10]:
english_tweets_raw.select('text').show(1)

+--------------------+
|                text|
+--------------------+
|Tonight: Wednesda...|
+--------------------+
only showing top 1 row



In [11]:
def get_sentiment_score(text):
    sid_obj = SentimentIntensityAnalyzer() 
    sentiment_dict = sid_obj.polarity_scores(text) 
    return [sentiment_dict['neg']*100,sentiment_dict['neu']*100,sentiment_dict['pos']*100]

In [12]:
#define python functions as pysaprk sql
get_sentiment_score_udf=udf(lambda text: get_sentiment_score(text), ArrayType(FloatType()))

In [13]:
english_tweets_processed=english_tweets_raw.withColumn('sentimentscore',get_sentiment_score_udf('text'))

In [14]:
english_tweets_processed.show(50,truncate=False)

+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|status_id          |text                                                                                                                                                                                                                                                                                                                |sentimentscore    |
+-------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:

def get_negative_score(array):
    return array[0]

def get_neutral_score(array):
    return array[1]

def get_positive_score(array):
    return array[2]




In [16]:
#define python functions as pysaprk sql
get_negative_score_udf=udf(lambda array: get_negative_score(array), FloatType())
get_neutral_score_udf=udf(lambda array: get_neutral_score(array), FloatType())
get_positive_score_udf=udf(lambda array: get_positive_score(array), FloatType())

In [17]:
english_tweets_processed=english_tweets_processed.withColumn('negative_score',get_negative_score_udf('sentimentscore'))
english_tweets_processed=english_tweets_processed.withColumn('neutral_score',get_neutral_score_udf('sentimentscore'))
english_tweets_processed=english_tweets_processed.withColumn('positive_score',get_positive_score_udf('sentimentscore'))

In [18]:
english_tweets_processed.show(50,truncate=False)

+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+--------------+-------------+--------------+
|status_id          |text                                                                                                                                                                                                                                                                                                                |sentimentscore    |negative_score|neutral_score|positive_score|
+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
def get_label(array):
    positive_score=array[2]
    negative_score=array[0]
    if positive_score >= negative_score:
        #positive
        return 1
    else:
        #negative
        return 0

In [20]:
#define python functions as pysaprk sql
get_label_udf=udf(lambda array: get_label(array), IntegerType())

In [21]:
english_tweets_processed=english_tweets_processed.withColumn('label',get_label_udf('sentimentscore'))

In [22]:
english_tweets_processed.show(50,truncate=False)

+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+--------------+-------------+--------------+-----+
|status_id          |text                                                                                                                                                                                                                                                                                                                |sentimentscore    |negative_score|neutral_score|positive_score|label|
+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
english_tweets_processed.select('label').groupby('label').count().show()

+-----+------+
|label| count|
+-----+------+
|    1|104622|
|    0| 62827|
+-----+------+



In [25]:
#top 10 positive sentences
english_tweets_processed.sort(desc("positive_score")).limit(10).show()

+-------------------+--------------------+-----------------+--------------+-------------+--------------+-----+
|          status_id|                text|   sentimentscore|negative_score|neutral_score|positive_score|label|
+-------------------+--------------------+-----------------+--------------+-------------+--------------+-----+
|            So yeah|              yeah?"|[0.0, 0.0, 100.0]|           0.0|          0.0|         100.0|    1|
|1237908954103021568|Confidence inspir...|[0.0, 14.1, 85.9]|           0.0|         14.1|          85.9|    1|
|1238213178393157632|"Please ""GOD"" s...|[0.0, 19.4, 80.6]|           0.0|         19.4|          80.6|    1|
|1238164303787819013|live laugh love #...|[0.0, 20.4, 79.6]|           0.0|         20.4|          79.6|    1|
|1237916361151533058|Wow. Wow. #COVID1...|[0.0, 20.8, 79.2]|           0.0|         20.8|          79.2|    1|
|1238244216456781826|Wow, just wow #Co...|[0.0, 20.8, 79.2]|           0.0|         20.8|          79.2|    1|
|

In [26]:
#top 10 negative sentences
english_tweets_processed.sort(desc("negative_score")).limit(10).show()

+-------------------+--------------------+-----------------+--------------+-------------+--------------+-----+
|          status_id|                text|   sentimentscore|negative_score|neutral_score|positive_score|label|
+-------------------+--------------------+-----------------+--------------+-------------+--------------+-----+
|1237907867774210048|FUCK FUCK FUCK FU...| [93.7, 6.3, 0.0]|          93.7|          6.3|           0.0|    0|
|1238244834718240771|Hell no  #Coronav...|[87.2, 12.8, 0.0]|          87.2|         12.8|           0.0|    0|
|    Italy reports 2|           016 dead"|[81.1, 18.9, 0.0]|          81.1|         18.9|           0.0|    0|
|1238093580683616256|SCARY SHIT. #Coro...|[80.5, 19.5, 0.0]|          80.5|         19.5|           0.0|    0|
|1238173590312402945|#Uncertain Strang...|[80.0, 20.0, 0.0]|          80.0|         20.0|           0.0|    0|
|1238252401875922946|Fear will kill #C...|[79.8, 20.2, 0.0]|          79.8|         20.2|           0.0|    0|
|

In [27]:
#top 10 neutral sentences
english_tweets_processed.sort(desc("neutral_score")).limit(10).show()

+-------------------+--------------------+-----------------+--------------+-------------+--------------+-----+
|          status_id|                text|   sentimentscore|negative_score|neutral_score|positive_score|label|
+-------------------+--------------------+-----------------+--------------+-------------+--------------+-----+
|1237891164738424833|Politicians overr...|[0.0, 100.0, 0.0]|           0.0|        100.0|           0.0|    1|
|1237891246321852416|Guyana has become...|[0.0, 100.0, 0.0]|           0.0|        100.0|           0.0|    1|
|1237891168198746117|Today, the @WHO d...|[0.0, 100.0, 0.0]|           0.0|        100.0|           0.0|    1|
|1237891137257406469|An update pertain...|[0.0, 100.0, 0.0]|           0.0|        100.0|           0.0|    1|
|1237891169796755456|Why would #Incred...|[0.0, 100.0, 0.0]|           0.0|        100.0|           0.0|    1|
|1237891054600241152|Watch this newly ...|[0.0, 100.0, 0.0]|           0.0|        100.0|           0.0|    1|
|